# Gathering of Bayesian evidence for the ELM model

In [6]:
import pickle
from pathlib import Path
import numpy as np
from periodictable import elements

In [7]:
import exfor_tools
import jitr

In [8]:
proton = (1, 1)
neutron = (1, 0)

In [9]:
Ca40 = (40, 20)
Ca48 = (48, 20)

Zr90 = (90, 40)
Zr92 = (92, 40)
Zr94 = (94, 40)
Zr96 = (96, 40)

Sn124 = (124, 50)
Sn122 = (122, 50)
Sn120 = (120, 50)
Sn118 = (118, 50)
Sn116 = (116, 50)

Ba138 = (138, 56)

Ce140 = (140, 58)

Sm144 = (144, 62)
Sm148 = (148, 62)

Pb208 = (208, 82)
Pb206 = (206, 82)

In [5]:
targets = [
    Ca40,
    Ca48,
    Zr90,
    Zr92,
    Zr94,
    Zr96,
    Sn116,
    Sn118,
    Sn120,
    Sn122,
    Sn124,
    Ba138,
    Ce140,
    Sm144,
    Sm148,
    Pb206,
    Pb208,
]

In [18]:
for t in targets:
    A,Z = t
    n_data_path = Path(f"./{elements[Z]}{A}_n_elastic.pkl")
    if n_data_path.is_file():
        with open(n_data_path, "rb") as f:
            entries = pickle.load(f)
            N = len(entries)
        print(f"Read {N} entries from {n_data_path}")
    p_data_path = Path(f"./{elements[Z]}{A}_p_elastic.pkl")
    if p_data_path.is_file():
        with open(p_data_path, "rb") as f:
            entries = pickle.load(f)
            N = len(entries)
        print(f"Read {N} entries from {p_data_path}")


Read 3 entries from Ca40_n_elastic.pkl
Read 2 entries from Ca40_p_elastic.pkl
Read 1 entries from Ca48_n_elastic.pkl
Read 2 entries from Ca48_p_elastic.pkl
Read 2 entries from Zr90_n_elastic.pkl
Read 2 entries from Zr90_p_elastic.pkl
Read 1 entries from Zr92_n_elastic.pkl
Read 2 entries from Zr92_p_elastic.pkl
Read 1 entries from Zr94_n_elastic.pkl
Read 2 entries from Zr94_p_elastic.pkl
Read 2 entries from Zr96_p_elastic.pkl
Read 2 entries from Sn116_n_elastic.pkl
Read 2 entries from Sn116_p_elastic.pkl
Read 1 entries from Sn118_n_elastic.pkl
Read 2 entries from Sn118_p_elastic.pkl
Read 2 entries from Sn120_n_elastic.pkl
Read 2 entries from Sn120_p_elastic.pkl
Read 1 entries from Sn122_n_elastic.pkl
Read 2 entries from Sn122_p_elastic.pkl
Read 2 entries from Sn124_n_elastic.pkl
Read 2 entries from Sn124_p_elastic.pkl
Read 2 entries from Ba138_p_elastic.pkl
Read 1 entries from Ce140_n_elastic.pkl
Read 2 entries from Ce140_p_elastic.pkl
Read 2 entries from Sm144_p_elastic.pkl
Read 2 entr

In [19]:
A,Z = 48,20
n_data_path = Path(f"./{elements[Z]}{A}_n_elastic.pkl")
n_data_path.is_file()

True

In [20]:
with open(n_data_path, "rb") as f:
    entries = pickle.load(f)

In [21]:
entries

{'14303': <exfor_tools.exfor_tools.ExforDifferentialData at 0x7e5c8d7f14d0>}